In [ ]:
from transformers import PaliGemmaForConditionalGeneration, AutoProcessor
from peft import PeftModel
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

base_model = "google/paligemma-3b-pt-896"
adapter_model = "arnaudstiegler/out"

model = PaliGemmaForConditionalGeneration.from_pretrained(base_model)
model = PeftModel.from_pretrained(model, adapter_model).to(device)
processor = AutoProcessor.from_pretrained(base_model)

model = model.eval()

In [ ]:
from datasets import load_dataset 

dataset = load_dataset("arnaudstiegler/synthetic_us_passports_easy")

In [ ]:
sample = dataset['train'][0]
texts = ["Process "]
images = [sample["image"].convert("RGB")]
tokens = processor(
    text=texts,
    images=images,
    return_tensors="pt",
    truncation=True,
    padding=True,
    max_length=128,
    tokenize_newline_separately=False,
)

In [ ]:
out = model.generate(input_ids=tokens['input_ids'].to('cuda'), attention_mask=tokens['attention_mask'].to('cuda'), pixel_values=tokens['pixel_values'].to('cuda'), max_new_tokens=128)

In [ ]:
print(processor.batch_decode(out))
print(sample)